# Integrated Test
In this test, we'll use a SageMaker Estimator (https://sagemaker.readthedocs.io/en/stable/estimators.html) to encapsulate the docker image published to ECR and start a local test, but this time, using the SageMaker library.

In [12]:
import sagemaker
import json
from sagemaker import get_execution_role

role = get_execution_role()
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
prefix='mlops/iris'

## Upload the dataset
In the previous exercise, prepared the training and validation dataset. Now, we'll upload the CSVs to S3 and share them with an Estimator

In [13]:
train_path = sagemaker_session.upload_data(path='input/data/train', key_prefix='iris-model/input/train')
test_path = sagemaker_session.upload_data(path='input/data/validation', key_prefix='iris-model/input/validation')
print("Train: %s\nValidation: %s" % (train_path, test_path) )

Train: s3://sagemaker-ap-south-1-523666378432/iris-model/input/train
Validation: s3://sagemaker-ap-south-1-523666378432/iris-model/input/validation


## And now, we can use a SageMaker Estimator for training and deploying the container we've created

In [14]:
# Create the estimator
# iris-model:test is the name of the container created in the previous notebook
# By the local codebuild test. An image with that name:tag was pushed to the ECR.

In [15]:
iris = sagemaker.estimator.Estimator('iris-model:test',
                                    role,
                                    instance_count=1, 
                                    instance_type='local',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix))
hyperparameters = {
    'max_depth': 20,
    'n_jobs': 4,
    'n_estimators': 120
}

print(hyperparameters)
iris.set_hyperparameters(**hyperparameters)

{'max_depth': 20, 'n_jobs': 4, 'n_estimators': 120}


### After you call .fit, a new training job will be executed inside the local Docker daemon and not in the SageMaker environment, on the cloud

In [16]:
iris.fit({"train":train_path, "validation":test_path})

Pulling algo-1-zqn1p (iris-model:test)...
The image for the service you're trying to recreate has been removed. If you continue, volume data could be lost. Consider backing up your data before continuing.

Continue with the new image? [yN]pull access denied for iris-model, repository does not exist or may require 'docker login': denied: requested access to the resource is denied


RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmpct02jzt6/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 1

### The next command will launch a new container in your local Docker daemon. Then you can use the returned predictor for testing it